In [24]:
library(sf)
library(rsi)
library(terra)
library(raster)

### Como construir um objeto raster no pacote terra

In [25]:
raster_t = rast(nrows = 6, ncols= 6, 
                xmin = -1.5, xmax = 1.5, ymin = -1.5, ymax = 1.5, 
                vals = runif(36))

### Como construir um objeto raster no pacote raster

In [26]:
raster_r = raster(nrows = 6, ncols = 6, 
                  xmn = -1.5, xmx = 1.5, ymn = -1.5, ymx = 1.5,
                  vals = runif(36), crs = 4326)

### Obtendo um objeto raster de uma imagem de satélite landsat usando o pacote rsi.
### Obtenção de uma imagem de manaus usando um buffer circular

In [27]:
manaus = st_point(c(-59.9825, -3.1316)) ##--(lat,long) de manaus
manaus = st_sfc(manaus, crs = "EPSG:4326") ##--localização de manaus no crs padrão do R--##
manaus = st_buffer(st_transform(manaus, crs = "EPSG:31980"), 100) ##--criação de um buffer sobre manaus nao crs apropriado

### Código para obtenção da imagem usando o pacote rsi

In [ ]:
manaus_landsat = get_landsat_imagery(
    manaus,
    start_date = "2023-09-01",
    end_date = "2023-09-30",
    output_filename = tempfile(fileext = "manaus_landsat01.tif")
)

### Fim da obtenção da imagem usando buffer
### Obtenção de uma imagem de manaus usando um bbox

In [9]:
manaus <- sf::st_bbox(
  c(xmin = 820599.5, ymin = 9649968, xmax = 854014.7, ymax = 9677464),
  crs = 31980
)
class(manaus)

[1] "bbox"

In [10]:
manaus <- st_as_sf(st_as_sfc(manaus))
## manaus <- st_as_sfc(manaus) %>% st_as_sf()
class(manaus)

[1] "sf"         "data.frame"

In [15]:
manaus_landsat <- get_landsat_imagery(
  manaus,
  start_date = "2024-09-01",
  end_date = "2024-09-30",
  output_filename = tempfile(fileext = "img_002.tif")
)
manaus_landsat ## veja onde a imagem foi armazenada no seu computador

ERROR: Error: HTTP content type response 'text/plain' not defined for this operation.


In [17]:
# debug de api, erro de html geralmente é erro de autenticação
# Dentro da função download_function ou query_function
library(rstac)
library(httr)

# Define o AOI (Área de Interesse)
aoi_bbox <- sf::st_bbox(manaus)

# Faz a consulta à API
items <- rstac::stac_search(
    rstac::stac("https://planetarycomputer.microsoft.com/api/stac/v1/"),
    collections = "landsat-c2-l2",
    bbox = c(aoi_bbox["xmin"], aoi_bbox["ymin"], aoi_bbox["xmax"], aoi_bbox["ymax"]),
    datetime = "2024-09-01/2024-09-30",
    limit = 10
)

# Verifica o conteúdo da resposta
print(items)



Attaching package: ‘rstac’


The following object is masked from ‘package:rsi’:

    sign_planetary_computer




###rstac_query
- url: https://planetarycomputer.microsoft.com/api/stac/v1/
- params:
  - collections: landsat-c2-l2
  - datetime: 2024-09-01/2024-09-30
  - bbox: 820599.5,9649968,854014.7,9677464
  - limit: 10
- field(s): version, base_url, endpoint, params, verb, encode


In [19]:
# Primeiro, vamos transformar as coordenadas para WGS84 (EPSG:4326)
manaus_wgs84 <- sf::st_transform(manaus, 4326)
bbox_wgs84 <- sf::st_bbox(manaus_wgs84)

# Agora vamos tentar a consulta novamente
manaus_landsat <- get_landsat_imagery(
    manaus_wgs84,  # usando o objeto transformado
    start_date = "2024-09-01",
    end_date = "2024-09-30",
    output_filename = tempfile(fileext = "003.tif")
)

Warning message:
“The default pixel size arguments are intended for use with projected AOIs, but `aoi` appears to be in geographic coordinates.
ℹ Pixel X size: 30. Pixel Y size: 30.
ℹ These dimensions will be interpreted in the same units as `aoi` (likely degrees), which may cause errors.”


### Veja onde a imagem foi armazenada no seu computador

In [20]:
manaus_landsat

[1] "/tmp/RtmpZ4ZY3A/filebe6359e6ffaf003.tif"

### leitura da imagem armazenada no pc

In [21]:
manaus_landsat <- "/tmp/RtmpZ4ZY3A/filebe6359e6ffaf003.tif"

### explorando a imagem com o pacote raster

In [22]:
manaus_landsat_layer <- raster::raster(manaus_landsat)
manaus_landsat_layer
manaus_landsat_rast <- raster::brick(manaus_landsat)

class      : RasterLayer 
band       : 1  (of  8  bands)
dimensions : 1, 1, 1  (nrow, ncol, ncell)
resolution : 30, 30  (x, y)
extent     : -60.11637, -30.11637, -32.91353, -2.913532  (xmin, xmax, ymin, ymax)
crs        : +proj=longlat +datum=WGS84 +no_defs 
source     : filebe6359e6ffaf003.tif 
names      : A 


### selecionando bandas específicas usando a função subset ou via indexação

In [23]:
sub_im1 <- subset(manaus_landsat_rast, 1:3)
sub_im2 <- manaus_landsat_rast[[1:3]]